In [12]:
import gym
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
import numpy as np
import copy
import time
import random
from collections import deque
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
#env =FrozenLakeEnv(is_slippery=False,map_name="8x8")
env.render()


SFFF
FHFH
FFFH
HFFG


In [13]:
class Net(nn.Module):
    def __init__(self,para,bias=True):
        super(Net,self).__init__()
        self.input_dim=para['input_dim']
        self.output_dim=para['output_dim']
        self.hidden_dim=para['hidden_dim']
        
        self.w1=Parameter(torch.FloatTensor(self.input_dim,self.hidden_dim))
        self.b1=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w1)
#        nn.init.kaiming_uniform_(self.w1)
        nn.init.uniform_(self.b1,-0.001,0.001)

        self.w2=Parameter(torch.FloatTensor(self.hidden_dim,self.hidden_dim))
        self.b2=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w2)
#        nn.init.kaiming_uniform_(self.w2)
        nn.init.uniform_(self.b2,-0.001,0.001)
        
        self.w3=Parameter(torch.FloatTensor(self.hidden_dim,self.hidden_dim))
        self.b3=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w3)
#        nn.init.kaiming_uniform_(self.w3)
        nn.init.uniform_(self.b3,-0.001,0.001)
        
        self.w4=Parameter(torch.FloatTensor(self.hidden_dim,self.output_dim))
        self.b4=Parameter(torch.FloatTensor(self.output_dim))
        nn.init.kaiming_normal_(self.w4)
#        nn.init.kaiming_uniform_(self.w4)
        nn.init.uniform_(self.b4,-0.001,0.001)
        
    def forward(self,X):
        
        H=torch.relu( torch.einsum('li,ij->lj',X,self.w1)+self.b1)
        H=torch.relu( torch.einsum('li,ij->lj',H,self.w2)+self.b2)
        H=torch.relu( torch.einsum('li,ij->lj',H,self.w3)+self.b3)
        Y=torch.einsum('li,ij->lj',H,self.w4)+self.b4

        return Y

    def copy_val(self,Net2):
        self.w1.data.copy_(Net2.w1)
        self.w2.data.copy_(Net2.w2)
        self.w3.data.copy_(Net2.w3)
        self.w4.data.copy_(Net2.w4)
        self.b1.data.copy_(Net2.b1)
        self.b2.data.copy_(Net2.b2)
        self.b3.data.copy_(Net2.b3)
        self.b4.data.copy_(Net2.b4)

In [14]:
input_dim=env.observation_space.n
output_dim=env.action_space.n
hidden_dim=100
lr=0.001
lr2=1.0

para={'input_dim': input_dim, 'output_dim': output_dim, 'hidden_dim': hidden_dim}
model=Net(para)
model0=Net(para)
model0.copy_val(model)
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr)

In [20]:
max_episodes=300
max_step=100
gamma=0.998
epsilon0 = 1.0

epsilon=epsilon0
min_step=max_step
min_step_episode=max_episodes
min_step_path=[]
max_replay=1000
replay=deque(maxlen=max_replay)
#replay=[]
replay0=deque(maxlen=max_replay)
replay1=deque(maxlen=max_replay)
replay2=deque(maxlen=max_replay)
i_step=0
batch_size=50

for episode in range(max_episodes):
    epsilon=epsilon*0.99
    state = env.reset()
    total_reward = 0
    state_list = [state]
    win=0
    
    state_m=np.zeros([1,env.observation_space.n],dtype=np.float32)
    state_m[0][state]=1.0
    state_m_torch=torch.from_numpy(state_m)
    Q_pred=model0(state_m_torch)
    Q_pred_np=Q_pred.cpu().detach().numpy()
    if np.random.rand()>epsilon :
        action=np.argmax(Q_pred_np[0])
    else:
        action=np.random.choice(env.action_space.n) 
    for step in range(1,max_step+1):
        
        i_step+=1
   
        state_next,reward,game_over,_ = env.step(action)
        
        if reward==0 : 
            reward=-0.01
        if reward!=1.0 and game_over==1:
            reward=-1
        
        state_m_next=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m_next[0][state_next]=1.0
        state_m_next_torch=torch.from_numpy(state_m_next)
        Q_pred_next=model0(state_m_next_torch)
        Q_pred_next=Q_pred_next.cpu().detach().numpy()
        if np.random.rand()>epsilon :
            action_next=np.argmax(Q_pred_next[0])
        else:
            action_next=np.random.choice(env.action_space.n)
        
        #print(Q_pred_np)
        Y_true=copy.deepcopy(Q_pred_np[0])
        if reward==1.0 and game_over==1:
            Y_true[action]= reward
        else :
            Y_true[action]=reward+gamma* (Q_pred_next[0][action_next])
        Y_true=np.reshape(Y_true,[1,env.action_space.n])

        replay.append([state_m[0],Y_true])

#        if state in replay0:
#            index=replay0.index(state)
#            replay[index]=[state_m[0],Y_true]
#        else:
#            replay0.append(state)
#            replay.append([state_m[0],Y_true]) 
        
#            game_over=0
        # Q-Learning
        if i_step%50==0 and i_step!=0 and len(replay)>batch_size:
#            print(replay0)
            mini_batch = random.sample(replay,batch_size)
            states=np.zeros([batch_size,input_dim],dtype=np.float32)
            Qs=np.zeros([batch_size,output_dim],dtype=np.float32)
            for m in range(batch_size):
                states[m]=mini_batch[m][0]
                Qs[m]=mini_batch[m][1]
            
            for i in range(0,1):
                optimizer.zero_grad()
                states_torch=torch.from_numpy(states)
                Q_pred=model(states_torch)
                Qs_torch=torch.from_numpy(Qs)
                loss=criterion(Q_pred,Qs_torch)
                loss.backward(retain_graph=True)
                optimizer.step()
                cost=loss.data
            Q_pred_np=Q_pred.cpu().detach().numpy()


        if i_step%100==0  :
#            print(i_step,"update")
            model0.copy_val(model)

            
        state=state_next
        state_m=state_m_next
        action=action_next
        
        total_reward=total_reward+reward
        state_list.append(state)
        if game_over and reward==1:
            win=1
            if step< min_step:
                min_step=step
                min_step_path=state_list
                min_step_episode=episode
        if game_over==1:
            break
            

    line_out="Episode: %d, step: %d, total_reward: %f, epsilon: %f" %(episode, step, total_reward,epsilon)
    print(line_out)
    if reward==1 :
        print(state_list)
print( "min step: ", min_step, "min step episode", min_step_episode)
print("shortest path: ", min_step_path)
#print(Q)

Episode: 0, step: 6, total_reward: -1.050000, epsilon: 0.990000
Episode: 1, step: 15, total_reward: -1.140000, epsilon: 0.980100
Episode: 2, step: 10, total_reward: -1.090000, epsilon: 0.970299
Episode: 3, step: 8, total_reward: -1.070000, epsilon: 0.960596
Episode: 4, step: 2, total_reward: -1.010000, epsilon: 0.950990
Episode: 5, step: 17, total_reward: -1.160000, epsilon: 0.941480
Episode: 6, step: 5, total_reward: -1.040000, epsilon: 0.932065
Episode: 7, step: 6, total_reward: -1.050000, epsilon: 0.922745
Episode: 8, step: 18, total_reward: -1.170000, epsilon: 0.913517
Episode: 9, step: 4, total_reward: -1.030000, epsilon: 0.904382
Episode: 10, step: 3, total_reward: -1.020000, epsilon: 0.895338
Episode: 11, step: 3, total_reward: -1.020000, epsilon: 0.886385
Episode: 12, step: 9, total_reward: -1.080000, epsilon: 0.877521
Episode: 13, step: 34, total_reward: -1.330000, epsilon: 0.868746
Episode: 14, step: 2, total_reward: -1.010000, epsilon: 0.860058
Episode: 15, step: 11, total_r

Episode: 126, step: 19, total_reward: 0.820000, epsilon: 0.279042
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 8, 9, 13, 14, 13, 14, 15]
Episode: 127, step: 37, total_reward: -1.360000, epsilon: 0.276252
Episode: 128, step: 63, total_reward: -1.620000, epsilon: 0.273489
Episode: 129, step: 21, total_reward: -1.200000, epsilon: 0.270754
Episode: 130, step: 28, total_reward: -1.270000, epsilon: 0.268047
Episode: 131, step: 20, total_reward: -1.190000, epsilon: 0.265366
Episode: 132, step: 46, total_reward: -1.450000, epsilon: 0.262713
Episode: 133, step: 6, total_reward: -1.050000, epsilon: 0.260085
Episode: 134, step: 66, total_reward: -1.650000, epsilon: 0.257485
Episode: 135, step: 52, total_reward: -1.510000, epsilon: 0.254910
Episode: 136, step: 2, total_reward: -1.010000, epsilon: 0.252361
Episode: 137, step: 21, total_reward: -1.200000, epsilon: 0.249837
Episode: 138, step: 31, total_reward: -1.300000, epsilon: 0.247339
Episode: 139, step: 50, total_reward: 0.510000, epsilon: 0.244865


Episode: 243, step: 33, total_reward: -1.320000, epsilon: 0.086097
Episode: 244, step: 100, total_reward: -1.000000, epsilon: 0.085236
Episode: 245, step: 9, total_reward: -1.080000, epsilon: 0.084384
Episode: 246, step: 32, total_reward: -1.310000, epsilon: 0.083540
Episode: 247, step: 50, total_reward: 0.510000, epsilon: 0.082704
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 9, 13, 14, 15]
Episode: 248, step: 24, total_reward: -1.230000, epsilon: 0.081877
Episode: 249, step: 100, total_reward: -1.000000, epsilon: 0.081059
Episode: 250, step: 54, total_reward: -1.530000, epsilon: 0.080248
Episode: 251, step: 70, total_reward: -1.690000, epsilon: 0.079445
Episode: 252, step: 23, total_reward: -1.220000, epsilon: 0.078651
Episode: 253, step: 100, total_reward: -1.000000, epsilon: 0.077864
Episode: 254, step: 25, total_reward: -1.240000, epsilon: 0.077086
Episode: 255, step: 13, total_reward: 

In [11]:
#state=np.arange(0,15)
#sess.run(init)
state_m=np.zeros([env.observation_space.n,env.observation_space.n],dtype=np.float32)
for i in range(0,env.observation_space.n):
    state_m[i][i]=1.0
print(state_m)
state_m_torch=torch.from_numpy(state_m)
Q_pred=model(state_m_torch)
Q_pred=Q_pred.cpu().detach().numpy()
print(Q_pred)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[-0.23958032 -0.28155452 -0.2315142  -0.2466275 ]
 [-0.24987632 -0.29821908 -0.19079159 -0.23576882]
 [-0.2347972   0.32604674 -0.23196468 -0.23123698]
 [-0.23416203 -0.15894885 -0.1

In [13]:
a=torch.rand([10])

In [14]:
b=torch.zeros([10])

In [15]:
print(a,b)

tensor([0.0485, 0.8422, 0.3865, 0.9593, 0.0383, 0.7846, 0.7052, 0.0029, 0.7384,
        0.0160]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [16]:
a.copy_(b)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
b

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
a

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
a[5]=2

In [21]:
a

tensor([0., 0., 0., 0., 0., 2., 0., 0., 0., 0.])

In [22]:
b

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [92]:
w1=Parameter(torch.FloatTensor(10))
w2=Parameter(torch.FloatTensor(10))

In [93]:
print(w1,w2)

Parameter containing:
tensor([-5.3691e-26,  4.5588e-41, -5.3998e-26,  4.5588e-41, -5.3680e-26,
         4.5588e-41, -5.3685e-26,  4.5588e-41, -5.3689e-26,  4.5588e-41],
       requires_grad=True) Parameter containing:
tensor([-2.9900e+08,  3.0660e-41,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       requires_grad=True)


In [94]:
w1.data.copy_(w2)

tensor([-2.9900e+08,  3.0660e-41,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       grad_fn=<CopyBackwards>)

In [95]:
w1[4]=6
print(w1)
print(w2)

Parameter containing:
tensor([-2.9900e+08,  3.0660e-41,  0.0000e+00,  0.0000e+00,  6.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       grad_fn=<CopySlices>)
Parameter containing:
tensor([-2.9900e+08,  3.0660e-41,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       requires_grad=True)


In [100]:
input_dim=env.observation_space.n
output_dim=env.action_space.n
hidden_dim=10

para={'input_dim': input_dim, 'output_dim': output_dim, 'hidden_dim': hidden_dim}
m=Net(para)
m2=Net(para)

In [86]:
m3=m2.

In [90]:
m.copy_val(m2)

Net()

TypeError: clone(): argument 'input' (position 1) must be Tensor, not Net